<a href="https://colab.research.google.com/github/refercon/ipynb/blob/main/Face_recognition_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mtcnn


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
      --------------------------------------- 0.0/2.3 MB 660.6 kB/s eta 0:00:04
     -- ------------------------------------- 0.1/2.3 MB 1.4 MB/s eta 0:00:02
     ------ --------------------------------- 0.4/2.3 MB 2.9 MB/s eta 0:00:01
     ----------- ---------------------------- 0.7/2.3 MB 3.8 MB/s eta 0:00:01
     ------------------ --------------------- 1.0/2.3 MB 4.6 MB/s eta 0:00:01
     --------------------------------- ------ 1.9/2.3 MB 7.1 MB/s eta 0:00:01
     ---------------------------------------- 2.3/2.3 MB 7.6 MB/s eta 0:00:00


In [ ]:
# import cv2
# from facenet_pytorch import MTCNN
# import numpy as np
# import time
# import random

# # 创建一个MTCNN实例
# mtcnn = MTCNN()
# # 打开摄像头
# cap = cv2.VideoCapture(0)
# # 初始化人脸计数器
# face_count = 0

# while face_count < 500:
#     # 读取视频帧
#     ret, frame = cap.read()
#     # 使用MTCNN检测人脸
#     boxes, _ = mtcnn.detect(frame)
#     # 如果检测到人脸
#     if boxes is not None:
#         for box in boxes:
#             # 提取人脸坐标
#             x1, y1, x2, y2 = box.astype(int)
#             # 提取人脸图像
#             face = frame[y1:y2, x1:x2]
#             # 数据增强：缩放
#             face = cv2.resize(face, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_CUBIC)
#             # 数据增强：随机旋转
#             angle = random.randint(-30, 30)  # 随机选择旋转角度
#             rows, cols = face.shape[:2]
#             M = cv2.getRotationMatrix2D(((cols-1)/2.0, (rows-1)/2.0), angle, 1)
#             face = cv2.warpAffine(face, M, (cols, rows))
#             # 生成唯一的文件名
#             filename = f"E:/pic/1/face_{face_count}.png"
#             # 保存人脸图像
#             cv2.imwrite(filename, face)
#             # 在视频帧上画出人脸边框
#             cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
#             # 更新人脸计数器
#             face_count += 1

#     # 显示带有人脸边框的视频帧
#     cv2.imshow('Video', frame)

#     # 按'q'键退出循环
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # 释放摄像头并关闭所有窗口
# cap.release()
# cv2.destroyAllWindows()


In [ ]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

class FaceDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = []
        self.labels = []

        for label in [1, 2]:
            directory = os.path.join(root_dir, str(label))
            for filename in os.listdir(directory):
                if filename.endswith('.png'):
                    self.images.append(os.path.join(directory, filename))
                    self.labels.append(label - 1)  # labels: 0 or 1

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.open(self.images[idx])
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# 数据转换
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor()
])

# 创建数据集
dataset = FaceDataset(root_dir='E:/pic', transform=transform)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)
    # print(labels)

In [ ]:
# import matplotlib.pyplot as plt

# # 只显示一批图像
# images, labels = next(iter(dataloader))
# plt.figure(figsize=(12, 8))
# for i in range(2):
#     plt.subplot(1, 2, i + 1)
#     plt.imshow(images[i].permute(1, 2, 0))  # 将通道顺序改为(H, W, C)以适应matplotlib
#     plt.title(f"Label: {labels[i]}")
# plt.show()


In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)
        # 根据新的特征图尺寸调整全连接层
        self.fc1 = nn.Linear(32 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 3)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        # 删除conv3层
        x = x.view(-1, 32 * 8 * 8)
        x = F.relu(self.fc1(self.dropout(x)))
        x = self.fc2(x)
        return x

model = SimpleCNN()





In [ ]:

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
num_epochs = 10
from sklearn.metrics import f1_score, accuracy_score
import numpy as np

def calculate_accuracy_f1(model, dataloader, device):
    model.eval()  # 设置模型为评估模式
    true_labels = []
    pred_labels = []
    with torch.no_grad():
        for data in dataloader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)  # 将数据转移到设备
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            true_labels.extend(labels.cpu().numpy())  # 将标签移回CPU
            pred_labels.extend(predicted.cpu().numpy())  # 将预测结果移回CPU

    accuracy = accuracy_score(true_labels, pred_labels)
    f1 = f1_score(true_labels, pred_labels, average='weighted')
    return accuracy, f1

# 训练过程
for epoch in range(num_epochs):
    model.train()  # 设置模型为训练模式
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)  # 将数据转移到设备
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # 计算每个epoch的平均loss
    epoch_loss = running_loss / len(dataloader)

    # 计算accuracy和f1-score
    accuracy, f1 = calculate_accuracy_f1(model, dataloader, device)

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.4f}, F1-Score: {f1:.4f}")
# 保存训练好的模型
torch.save(model.state_dict(), 'face_classification_model.pth')

Epoch 1/10, Loss: 0.4824, Accuracy: 0.9900, F1-Score: 0.9900
Epoch 2/10, Loss: 0.0519, Accuracy: 0.9900, F1-Score: 0.9900
Epoch 3/10, Loss: 0.0615, Accuracy: 0.9770, F1-Score: 0.9770
Epoch 4/10, Loss: 0.0892, Accuracy: 0.9570, F1-Score: 0.9570
Epoch 5/10, Loss: 0.0206, Accuracy: 0.9790, F1-Score: 0.9790
Epoch 6/10, Loss: 0.0809, Accuracy: 0.9300, F1-Score: 0.9297
Epoch 7/10, Loss: 0.0782, Accuracy: 0.9060, F1-Score: 0.9057
Epoch 8/10, Loss: 0.1076, Accuracy: 0.9430, F1-Score: 0.9430
Epoch 9/10, Loss: 0.0096, Accuracy: 0.8760, F1-Score: 0.8760
Epoch 10/10, Loss: 0.0073, Accuracy: 0.9720, F1-Score: 0.9720


In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image

# 加载模型
model = SimpleCNN()
model.load_state_dict(torch.load('face_classification_model.pth'))
model.eval()

# 定义转换
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 加载图像
img = Image.open('E:\\pic\\2\\face_200.png')  # 请将'your_image.jpg'替换为你的图像文件名

# 应用转换并添加批处理维度
input = transform(img).unsqueeze(0)

# 进行预测
output = model(input)

# 获取预测结果
_, predicted = torch.max(output.data, 1)

print('Predicted label:', predicted.item())


Predicted label: 1


In [ ]:
import cv2
import torch
import torchvision.transforms as transforms
from PIL import Image
from facenet_pytorch import MTCNN

# 加载模型
model = SimpleCNN()
model.load_state_dict(torch.load('face_classification_model.pth'))
model.eval()

# 定义转换
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
# 定义标签列表
labels = ['YC', 'WW']  # 请根据你的模型替换这些标签
# 创建MTCNN模型
mtcnn = MTCNN()

# 打开摄像头
cap = cv2.VideoCapture(0)

confidence_threshold = 0.9  # 置信度阈值

while True:
    # 读取帧
    ret, frame = cap.read()
    if not ret:
        break

    # 使用MTCNN进行人脸检测
    boxes, probs = mtcnn.detect(frame)

    # 如果检测到人脸
    if boxes is not None:
        for box, prob in zip(boxes, probs):
            # 检查置信度

                # 提取人脸
            face = frame[int(box[1]):int(box[3]), int(box[0]):int(box[2])]
                # 转换帧为图像
            img = Image.fromarray(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))

            # 应用转换并添加批处理维度
            input_tensor = transform(img).unsqueeze(0)

            # 进行预测
            output = model(input_tensor)

                # 获取预测结果
            _, predicted = torch.max(output.data, 1)

                # 获取预测标签
            label = labels[predicted.item()] if predicted.item() < len(labels) else 'Unknown'

            # 在帧上添加预测标签
            cv2.putText(frame, label, (int(box[0]), int(box[1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # 在帧上画出人脸的边界框
            cv2.rectangle(frame, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0, 255, 0), 2)

    # 显示帧
    cv2.imshow('Video', frame)

    # 按'q'键退出
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 释放摄像头并关闭窗口
cap.release()
cv2.destroyAllWindows()


d:\Anaconda3\lib\site-packages\torch\nn\functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"
